In [5]:
import pandas as pd
import numpy as np

In [53]:
train = pd.read_csv("train.csv")
train.sample(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
43,44,1,2,"Laroche, Miss. Simonne Marie Anne Andree",female,3.0,1,2,SC/Paris 2123,41.5792,NaN,C
592,593,0,3,"Elsbury, Mr. William James",male,47.0,0,0,A/5 3902,7.2500,NaN,S


In [58]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [57]:
train.Age.fillna(train.Age.mean(), inplace = True)

In [60]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,13.002015,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,29.699118,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [73]:
count = 0
for i in train.duplicated():
    if i:
        count += 1
print(count)

0


In [75]:
train["children"] = train.Age.apply(lambda x: 1 if x < 15 else 0)

In [80]:
train.sample(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,children
874,875,1,2,"Abelson, Mrs. Samuel (Hannah Wizosky)",female,28.0,1,0,P/PP 3381,24.00,NaN,C,0
535,536,1,2,"Hart, Miss. Eva Miriam",female,7.0,0,2,F.C.C. 13529,26.25,NaN,S,1


In [81]:
train = pd.get_dummies(train, columns=["Sex", "Pclass"])

In [83]:
train.sample(2)

,PassengerId,Survived,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,children,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
625,626,0,"Sutton, Mr. Frederick",61.000000,0,0,36963,32.3208,D50,S,0,0,1,1,0,0
359,360,1,"Mockler, Miss. Helen Mary ""Ellie""",29.699118,0,0,330980,7.8792,NaN,Q,0,1,0,0,0,1


In [85]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [87]:
y_columns = ["Survived"]

x_columns = ["Age", "SibSp", "Parch", "Fare", "children",
             "Sex_female", "Pclass_1", "Pclass_2"]

In [96]:
x_train,x_val, y_train, y_val = train_test_split(train[x_columns], 
                                                 train[y_columns],
                                                 test_size=0.2,
                                                 random_state=42)

In [97]:
model = RandomForestClassifier(n_estimators=100,
                              max_depth=5,
                              random_state=42,
                              oob_score = True,
                              criterion = "entropy")

clf = model.fit(x_train, np.ravel(y_train))

In [98]:
clf.score(x_val, y_val)

0.7988826815642458

In [100]:
print(pd.crosstab(np.array(y_val).reshape(1,-1)[0], 
                  clf.predict(x_val) ,
                  rownames=['Actual'], 
                  colnames=['Predicted'],
                  margins=True))

Predicted    0   1  All
Actual                 
0           92  13  105
1           23  51   74
All        115  64  179


In [101]:
cross_val_score(estimator=clf, 
                X=(x_train),
                y= y_train.values.ravel(), 
                cv=3,
                n_jobs=-1,
                verbose = 1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    2.1s finished


array([ 0.81932773,  0.83122363,  0.79324895])

In [102]:
from sklearn.feature_selection import RFECV

selector = RFECV(clf, step=1, cv=5, n_jobs=-1)
selector = selector.fit(x_train, y_train)
print(pd.DataFrame(selector.ranking_, index = x_columns, columns=['importance']).sort_values('importance', ascending=True))

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


            importance
Age                  1
SibSp                1
Parch                1
Fare                 1
children             1
Sex_female           1
Pclass_1             1
Pclass_2             1


In [105]:
from sklearn.model_selection import GridSearchCV

a  = GridSearchCV(clf, {'n_estimators':[100,200,300],
                        'max_depth': [2,4,6,7]}, cv = 4)
# a.fit(x_train, np.array(y_train).reshape(1,712)[0])
# a.fit(x_train, np.array(y_train).reshape(1,-1)[0])

a.fit(x_train, np.array(np.ravel(y_train)).reshape(1,-1)[0])

GridSearchCV(cv=4, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=True, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_estimators': [100, 200, 300], 'max_depth': [2, 4, 6, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [106]:
a.best_params_

{'max_depth': 4, 'n_estimators': 200}

In [1]:
help()


Welcome to Python 3.6's help utility!

If this is your first time using Python, you should definitely check out
the tutorial on the Internet at https://docs.python.org/3.6/tutorial/.

Enter the name of any module, keyword, or topic to get help on writing
Python programs and using Python modules.  To quit this help utility and
return to the interpreter, just type "quit".

To get a list of available modules, keywords, symbols, or topics, type
"modules", "keywords", "symbols", or "topics".  Each module also comes
with a one-line summary of what it does; to list the modules whose name
or summary contain a given string such as "spam", type "modules spam".

help> modules

Please wait a moment while I gather a list of all available modules...



/opt/conda/lib/python3.6/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)
/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/opt/conda/lib/python3.6/site-packages/skimage/novice/__init__.py:103: UserWarning: The `skimage.novice` module was deprecated in version 0.14. It will be removed in 0.16.
  warnings.warn("The `skimage.novice` module was deprecated in version 0.14. "
/opt/conda/lib/python3.6/site-packages/skimage/viewer/__init__.py:6: UserWarning: Viewer requires Qt
  warn('Viewer requires Qt')
/opt/conda/lib/python3.6/pk

AG                  builtins            kiwisolver          select
Cython              bz2                 lib2to3             selectors
IPython             cProfile            linecache           send2trash
OpenSSL             calendar            llvmlite            setup
PIL                 certifi             locale              setuptools
__future__          cffi                logging             shelve
_ast                cgi                 lzma                shlex
_asyncio            cgitb               macpath             shutil
_bisect             chardet             macurl2path         signal
_blake2             chunk               mailbox             sip
_bootlocale         cloudpickle         mailcap             sipconfig
_bz2                cmath               mako                sipdistutils
_cffi_backend       cmd                 markupsafe          site
_codecs             code                marshal             six
_codecs_cn          codecs              math        

ModuleNotFoundError: No module named 'tensorflow'